# App

# Import Library
Kode dibawah digunakan untuk mengimport library yang diperlukan.

In [1]:
from nltk.tokenize import word_tokenize
import re
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
from itertools import groupby
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_selection import mutual_info_classif
from statistics import stdev
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# Import Dataset
Kode dibawah digunakan untuk membaca file dengan format .csv lalu nantinya ditampilkan dalam bentuk DataFrame.

In [2]:
TWEET_DATA = pd.read_csv("Dataset.csv", encoding= 'unicode_escape')
print(TWEET_DATA[['Tweet']].head(5))

                                               Tweet
0  [askrl] minta saran skincare buat pemula bgt d...
1                      @txtdrstoryWA Geblek ni orang
2  abis minum balik terus saur rasanya anjing kud...
3  Ini 4th gen it boy gak selese2, gedek jdny, je...
4        lagi enak enak puasa trus mens tuh tai bgtt


# Preprocessing
Preprocessing dilakukan untuk menghilangkan noise yang nantinya akan mengganggu saat proses pembelajaran metode machine learning. Adapun tahapan preprocessing yang dilakukan adalah sebagai berikut:

## Case Folding
Merupakan proses mengubah huruf pada dokumen teks menjadi huruf kecil. Kode dibawah digunakan untuk mengubah huruf alphabet menjadi huruf kecil.

In [3]:
def case_folding(text):
    return str.lower(text)
TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(case_folding)
print(TWEET_DATA[['Tweet']].head(5))

                                               Tweet
0  [askrl] minta saran skincare buat pemula bgt d...
1                      @txtdrstorywa geblek ni orang
2  abis minum balik terus saur rasanya anjing kud...
3  ini 4th gen it boy gak selese2, gedek jdny, je...
4        lagi enak enak puasa trus mens tuh tai bgtt


## Cleaning
Merupakan prosses menghapus karakter – karakter yang tidak diperlukan seperti URL, angka, simbol dan lain sebagainya. Kode dibawah terdapat fungsi yang mana didalamnya terdapat berbagai kode dari python berupa method untuk:
- menghapus teks selain kode ascii
- menghapus mention, link, hashtag
- menghapus angka
- menghapus huruf single
- menghapus huruf berulang
- menghapus tanda baca

In [4]:
def cleaning(text):
    #hapus teks yang bukan ascii
    text = text.encode('ascii', 'replace').decode('ascii')
    #hapus mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    #hapus url
    text = text.replace("http://", " ").replace("https://", " ")
    #hapus angka
    text = re.sub(r'[0-9]+', '', text)
    #hapus single huruf
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    #hapus huruf berulang
    text = ''.join(c for c, _ in groupby(text))
    #hapus tanda baca
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in text:
        if ele in punc:
            text = text.replace(ele, " ")   
    fixtext = re.sub('\s+',' ',text)
    return fixtext
TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(cleaning)
print(TWEET_DATA[['Tweet']].head(5))

                                               Tweet
0   askrl minta saran skincare buat pemula bgt do...
1                                    geblek ni orang
2  abis minum balik terus saur rasanya anjing kud...
3  ini th gen it boy gak selese gedek jdny jelas ...
4         lagi enak enak puasa trus mens tuh tai bgt


## Tokenisasi
Merupakan proses memecah kalimat menjadi bagian – bagian yang bermakna dan mengidentifikasi entitas individu dalam kalimat. Kode dibawah digunakan untuk mengubah kata dalam suatu teks menjadi token.

In [5]:
def tokenisasi(text):
    return word_tokenize(text)
TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(tokenisasi)
print(TWEET_DATA[['Tweet']].head(5))

                                               Tweet
0  [askrl, minta, saran, skincare, buat, pemula, ...
1                                [geblek, ni, orang]
2  [abis, minum, balik, terus, saur, rasanya, anj...
3  [ini, th, gen, it, boy, gak, selese, gedek, jd...
4  [lagi, enak, enak, puasa, trus, mens, tuh, tai...


## Normalisasi
Kata yang tidak baku, ambigu atau kata yang diulang – ulang akan diubah menjadi kata yang baku. Kode dibawah digunakan untuk menormalisasi teks dengan mencocokkannya pada sebuah file yang didalamnya terdapat daftar kata sebelum dan sesudah normalisasi.

In [6]:
normalizad_word = pd.read_excel("_normalisasi.xlsx")
normalizad_word_dict = {}
for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 
def normalisasi(document):
    kalimat = document
    for term in range(len(kalimat)):
        if kalimat[term] in normalizad_word_dict:
            kalimat[term] = normalizad_word_dict[kalimat[term]]
    hasil = " ".join(kalimat).split()
    return hasil
TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(normalisasi)
print(TWEET_DATA[['Tweet']].head(5))

                                               Tweet
0  [tanya, minta, saran, skincare, buat, pemula, ...
1                               [geblek, ini, orang]
2  [habis, minum, balik, terus, sahur, rasa, anji...
3  [ini, tahun, generasi, it, bang, tidak, selese...
4  [lagi, enak, enak, puasa, terus, mens, itu, ta...


## Stemming
Proeses mengubah kata menjadi kata akarnya. Kode dibawah digunakan untuk mengubah kata menjadi kata
akarnya dengan menggunakan bantuan library dari sastrawi.

In [7]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemmed_wrapper(term):
    return stemmer.stem(term)
term_dict = {}
for document in TWEET_DATA['Tweet']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
def stemming(document):
    return [term_dict[term] for term in document]
TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(stemming)
print(TWEET_DATA[['Tweet']].head(5))

                                               Tweet
0  [tanya, minta, saran, skincare, buat, mula, ba...
1                               [geblek, ini, orang]
2  [habis, minum, balik, terus, sahur, rasa, anji...
3  [ini, tahun, generasi, it, bang, tidak, selese...
4  [lagi, enak, enak, puasa, terus, mens, itu, ta...


## Stopword Removal
Merupakan proses untuk menghilangkan kata – kata yang sering muncul (stopword). Kode dibawah digunakan untuk menghapus stopword dari suatu teks dengan menggunakan library dari sastrawi. Teks "kamu","dia","ia","beliau" dikecualikan karena kata tersebut merupakan kata ganti orang ketiga dan biasanya terdapat pada teks yang mengandung kata bullying.

In [8]:
lsw_INA = set(stopwords.words('indonesian'))
lsw_INA_ex = set(("kamu","dia","ia","beliau"))
new_lsw = lsw_INA - lsw_INA_ex
def stopword_removal(text):
    return [word for word in text if not word in new_lsw]
TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(stopword_removal)
print(TWEET_DATA[['Tweet']].head(5))
print('')

                                               Tweet
0  [saran, skincare, banget, buta, skincare, merk...
1                                    [geblek, orang]
2        [habis, minum, sahur, anjing, sadar, entot]
3  [generasi, it, bang, selese, kesal, banget, ye...
4             [enak, enak, puasa, mens, tai, banget]



Kode dibawah digunakan untuk mengubah teks hasil preprocessing menjadi kalimat. Hal ini bertujuan agar memudahkan saat proses TF - IDF.

In [9]:
def join_text_list(texts):
    texts = ast.literal_eval(str(texts))
    a = ' '.join([text for text in texts])
    return a
TWEET_DATA["Tweet"] = TWEET_DATA["Tweet"].apply(join_text_list)
print(TWEET_DATA[['Tweet']].head(5))

                                               Tweet
0  saran skincare banget buta skincare merk recom...
1                                       geblek orang
2               habis minum sahur anjing sadar entot
3  generasi it bang selese kesal banget yeonjun y...
4                    enak enak puasa mens tai banget


# Pembobotan Kata
Merupakan proses penghitungan bobot tiap kata yang dicari pada setiap dokumen sehingga dapat diketahui ketersediaan dan kemiripan suatu kata di dalam dokumen. Pembobotan dilakukan karena metode machine learning tidak bisa memproses data berupa teks.

## TF - IDF
Merupakan salah satu metode untuk menghitung bobot kata. TF - IDF digunakan untuk menentukan nilai sebuah kata di dalam sebuah dokumen. Kode dibawah digunakan untuk menghitung nilai TF - IDF menggunakan library dari scikit-learn. Lalu hasil perhitungan ditampilkan pada sebuah dataframe.

In [10]:
vectorize = TfidfVectorizer(norm=None)
response = vectorize.fit_transform(TWEET_DATA['Tweet'])
features_names = vectorize.get_feature_names_out()
alist=response.todense()
newData = pd.DataFrame(alist,columns=features_names)
print(newData)

     abang  abiz  absen  abu   ac  acap  acara  acount  adab  adam  ...   yu  \
0      0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
1      0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
2      0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
3      0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
4      0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
..     ...   ...    ...  ...  ...   ...    ...     ...   ...   ...  ...  ...   
995    0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
996    0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
997    0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
998    0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   
999    0.0   0.0    0.0  0.0  0.0   0.0    0.0     0.0   0.0   0.0  ...  0.0   

          yuk  yuta   za  zahra  zaman 

# Seleksi Fitur
Merupakan proses yang melibatkan penghapusan fitur yang tidak relevan dan berulang untuk meningkatkan kinerja teknik machine learning.

## Information Gain
Information Gain digunakan untuk mengukur seberapa relavan suatu fitur. Fitur yang telah dihitung nilai Information Gainnya kemudian diurutkan berdasarkan nilai terbesar, lalu diseleksi menggunakan nilai threshold sebagai ambang batas. Namun untuk kode dibawah, fitur tidak akan diseleksi menggunakan nilai threshold. Karena setiap setelah tahap K - Fold Cross Validation fitur yang telah diurutkan akan dikurangi satu dari nilai Information Gain terkecil. Hal ini dilakukan hingga jumlah fitur tersisa satu. Pengurangan fitur secara berkala dilakukan untuk mencari kombinasi fitur yang paling baik. 

In [11]:
col_tweet = newData.iloc[:]
col_label = TWEET_DATA.iloc[:,-1]

#menerapkan information gain
IG_apply_skl = mutual_info_classif(col_tweet,col_label,random_state=0)
IG = pd.Series(IG_apply_skl)
IG.index = col_tweet.columns

#bulatkan 5 angka dibelakang koma
IG_R = round(IG,5)
print('\nNilai Information Gain\n',IG_R)

#sorting descend
tinggirendah=IG_R.sort_values(ascending=False)
tinggirendah1 = tinggirendah.index.tolist()

print('\nHasil Sorting\n',tinggirendah)

selected_columns = newData[tinggirendah1]
print(selected_columns)


Nilai Information Gain
 abang     0.00000
abiz      0.01034
absen     0.00000
abu       0.00573
ac        0.00000
           ...   
zaman     0.00000
zero      0.01358
zi        0.00000
zidane    0.00000
zodiak    0.00000
Length: 2277, dtype: float64

Hasil Sorting
 kamu      0.05865
patrol    0.05150
kenyat    0.05136
chika     0.04976
tolong    0.04947
           ...   
kejap     0.00000
kecoa     0.00000
kayak     0.00000
kaya      0.00000
zodiak    0.00000
Length: 2277, dtype: float64
         kamu  patrol  kenyat  chika  tolong  info  ovo  lau  otak  way  ...  \
0    0.000000     0.0     0.0    0.0     0.0   0.0  0.0  0.0   0.0  0.0  ...   
1    0.000000     0.0     0.0    0.0     0.0   0.0  0.0  0.0   0.0  0.0  ...   
2    0.000000     0.0     0.0    0.0     0.0   0.0  0.0  0.0   0.0  0.0  ...   
3    0.000000     0.0     0.0    0.0     0.0   0.0  0.0  0.0   0.0  0.0  ...   
4    0.000000     0.0     0.0    0.0     0.0   0.0  0.0  0.0   0.0  0.0  ...   
..        ...     ...    

# K - Fold Cross Validation
Merupakan salah satu dari jenis pengujian cross validation yang berfungsi untuk menilai kinerja proses sebuah metode algoritma dengan membagi sampel data secara acak dan mengelompokkan data tersebut sebanyak nilai k – fold. Kemudian salah satu kelompok k – fold tersebut akan dijadikan sebagai data testing sedangkan sisa kelompok yang lain akan dijadikan sebagai data training.

## Random Forest
Merupakan salah satu metode ensemble learning yang cara kerjanya menggabungkan beberapa pohon dari metode decision tree. Hasil prediksi tiap pohon dikumpulkan lalu digunakan majority voting untuk mendapatkan hasil akhir prediksi. Kode dibawah merupakan kode pengujian K - Fold Cross Validation yang didalamnya menggunakan metode machine learning Random Forest. Confusion Matrix juga digunakan untuk menguji kefektifan metode Random Forest.

In [ ]:
k=10
kf = KFold(n_splits=k)

model = RandomForestClassifier(random_state=0)
temp_cm_acc = []
temp_cm_prec = []
temp_cm_rec = []

TP = 0
FP = 0
TN = 0
FN = 0
all_TP = []
all_FP = []
all_TN = []
all_FN = []

atribut = len(selected_columns.columns)

for j in range(atribut,0,-1):
    temp_new_col_tweet=selected_columns.drop(selected_columns.columns[j:], axis=1)
    
    v_acc_score = []
    v_prec_score = []
    v_rec_score = []

    no=1

    temp_TP_1iter = []
    temp_FP_1iter = []
    temp_TN_1iter = []
    temp_FN_1iter = []
    for train_index , test_index in kf.split(temp_new_col_tweet):
        X_train , X_test = temp_new_col_tweet.iloc[train_index,:],temp_new_col_tweet.iloc[test_index,:]
        y_train , y_test = col_label[train_index] , col_label[test_index]
        
        model.fit(X_train,y_train)#train model

        pred_values = model.predict(X_test)#test model
    
        res = pd.DataFrame(columns=['Tweet_Split','Label_Split','Label_Prediksi'])
        res['Tweet_Split'] = TWEET_DATA['Tweet'].iloc[test_index]
        res['Label_Split'] = TWEET_DATA['Label'].iloc[test_index]
        res['Label_Prediksi'] = pred_values
       
        totaltr = X_train.shape[0]
        total = res.shape[0]
        
        for i in range(total):
            check = res.iloc[i]
            if(check.Label_Split=='CB' and check.Label_Prediksi=='CB'):
                TP+=1
            elif(check.Label_Split=='CB' and check.Label_Prediksi=='NCB'):
                FP+=1
            elif(check.Label_Split=='NCB' and check.Label_Prediksi=='NCB'):
                TN+=1
            elif(check.Label_Split=='NCB' and check.Label_Prediksi=='CB'):
                FN+=1

        temp_TP_1iter.append(TP)
        temp_FP_1iter.append(FP)
        temp_TN_1iter.append(TN)
        temp_FN_1iter.append(FN)
        try:
            accuracy = (TP+TN)/(TP+FP+TN+FN)
            precision = TP/(TP+FP)
            recall = TP/(TP+FN)
            
        except ZeroDivisionError:
            accuracy = 0
            precision = 0
            recall = 0
           
        res_accuracy = round(accuracy,2)
        accuracy100=res_accuracy*100
        
        res_precision = round(precision,2)
        precision100=res_precision*100
       
        res_recall = round(recall,2)
        recall100 = res_recall*100
        
        v_acc_score.append(accuracy100)
        v_prec_score.append(precision100)
        v_rec_score.append(recall100)
        
        no=no+1
        TP = 0
        FP = 0
        TN = 0
        FN = 0

    avg_acc_score = sum(v_acc_score)/k
    res_avg_acc_score = round(avg_acc_score,2)
    temp_cm_acc.append(res_avg_acc_score)
    
    avg_prec_score = sum(v_prec_score)/k
    res_avg_prec_score = round(avg_prec_score,2)
    temp_cm_prec.append(res_avg_prec_score)

    avg_rec_score = sum(v_rec_score)/k
    res_avg_rec_score = round(avg_rec_score,2)
    temp_cm_rec.append(res_avg_rec_score)

    print("\nRata - Rata Confusion Matrix Menggunakan",atribut,"atribut")
    print("Akurasi =",res_avg_acc_score)
    print("Presisi =",res_avg_prec_score)
    print("Recall =",res_avg_rec_score)
    

Kode dibawah merupakan kode untuk menampilkan rata - rata Confusion Matrix hasil K - Fold Cross Validation dengan pengurangan fitur secara berakala dalam tampilan grafis.

In [ ]:
best_acc = max(temp_cm_acc)
best_prec = max(temp_cm_prec)
best_rec = max(temp_cm_rec)

temp_cm_acc.reverse()
temp_cm_prec.reverse()
temp_cm_rec.reverse()

var_acc = temp_cm_acc.index(best_acc)+1
var_prec = temp_cm_prec.index(best_prec)+1
var_rec = temp_cm_rec.index(best_rec)+1

len_temp_cm_acc = len(temp_cm_acc)

length = len_temp_cm_acc/2
wide = len_temp_cm_acc/3

wew = [x+1 for x in range(len_temp_cm_acc)]

fig = plt.figure() 
fig.set_size_inches(20, 10,forward=True)
fig.set_dpi(100)

x_plot = np.array([x+1 for x in range(len_temp_cm_acc)])

plt.xlabel('Jumlah Atribut',fontsize=30)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

y_plot_acc = np.array(temp_cm_acc)
y_plot_prec = np.array(temp_cm_prec)
y_plot_rec = np.array(temp_cm_rec)

panjang=len(x_plot)
panjang1=panjang//5
cek1=x_plot[:panjang1]
cek2=x_plot[-panjang1:]


if(var_acc in cek1):
    sumbuX=var_acc+10
elif(var_acc in cek1):
    sumbuX=var_acc-10
else:
    sumbuX=var_acc
    
if(var_prec in cek1):
    sumbuX=var_prec+10
elif(var_prec in cek1):
    sumbuX=var_prec-10
else:
    sumbuX=var_prec
        
if(var_rec in cek1):
    sumbuX=var_rec+10
elif(var_rec in cek1):
    sumbuX=var_rec-10
else:
    sumbuX=var_rec
        

kalimat_acc = str(var_acc)+" Atribut: "+str(best_acc)+"%"
plt.annotate(kalimat_acc, xy =(var_acc, best_acc),
                xytext =(var_acc, best_acc+4),arrowprops=dict(facecolor='black', shrink=0.03),
                ha='center', va='bottom',fontsize=30)

kalimat_prec = str(var_prec)+" Atribut: "+str(best_prec)+"%"
plt.annotate(kalimat_prec, xy =(var_prec, best_prec),
                xytext =(var_prec, best_prec+4),arrowprops=dict(facecolor='black', shrink=0.03),
                ha='center', va='bottom',fontsize=30)

kalimat_rec = str(var_rec)+" Atribut: "+str(best_rec)+"%"
plt.annotate(kalimat_rec, xy =(var_rec, best_rec),
                xytext =(var_rec, best_rec+4),arrowprops=dict(facecolor='black', shrink=0.03),
                ha='center', va='bottom',fontsize=30)


plt.ylabel('Nilai',fontsize=30)
plt.plot(x_plot, y_plot_acc, color='c', label='Accuracy')
plt.plot(x_plot, y_plot_prec,color='m', label='Precision')
plt.plot(x_plot, y_plot_rec,color='y', label='Recall')


plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=30)
plt.grid()
plt.show()